# Manchester Area Price Change Map

This notebook reads `manc_data.csv` and computes the percentage change in average house price from 2024 to 2025 for each Manchester postcode area (e.g. M11, M12). The map shades each area green for increases and red for decreases with darker shades indicating larger changes. Clicking a marker shows the exact change and percentage.

In [ ]:
# Install pandas, folium and pgeocode if needed
import sys, subprocess
for pkg in ['pandas','folium','pgeocode']:
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

In [ ]:
import pandas as pd
import folium
import pgeocode
import ipywidgets as widgets
from IPython.display import display

cols=['transaction_id','price','date','postcode','property_type','old_new','duration','paon','saon','street','locality','town','district','county','ppd_cat','record']
df=pd.read_csv('manc_data.csv',names=cols,header=None)
df['price']=df['price'].astype(float)
df['date']=pd.to_datetime(df['date'])
df['year']=df['date'].dt.year
df['prefix']=df['postcode'].str.split().str[0]

years=sorted(df['year'].unique())

nomi=pgeocode.Nominatim('gb')
coord=df.groupby('prefix')['postcode'].first().to_dict()
prefix_coords={p:(nomi.query_postal_code(pc)['latitude'],nomi.query_postal_code(pc)['longitude']) for p,pc in coord.items()}

increase_start=(144,238,144)
increase_end=(0,100,0)
decrease_start=(255,204,204)
decrease_end=(139,0,0)

def interp(start,end,ratio):
    r=int(start[0]+(end[0]-start[0])*ratio)
    g=int(start[1]+(end[1]-start[1])*ratio)
    b=int(start[2]+(end[2]-start[2])*ratio)
    return f'#{r:02x}{g:02x}{b:02x}'

def trimmed_mean(series):
    if series.empty:
        return None
    q_low=series.quantile(0.1)
    q_high=series.quantile(0.9)
    trimmed=series[(series>=q_low)&(series<=q_high)]
    return trimmed.mean() if not trimmed.empty else None

def create_map(y1,y2):
    avg=df.groupby(['prefix','year'])['price'].apply(trimmed_mean).unstack()
    change=avg[y2]-avg[y1]
    percent=((avg[y2]-avg[y1])/avg[y1])*100
    inc_max=percent.max()
    dec_min=percent.min()
    m=folium.Map(location=[53.4808,-2.2426],zoom_start=10)
    for area,pct in percent.dropna().items():
        lat,lon=prefix_coords.get(area,[53.4808,-2.2426])
        diff=change.get(area,0)
        if pct>=0:
            ratio=0 if inc_max==0 else pct/inc_max
            colour=interp(increase_start,increase_end,ratio)
        else:
            ratio=0 if dec_min==0 else pct/dec_min
            colour=interp(decrease_start,decrease_end,ratio)
        folium.CircleMarker(
            location=[lat,lon],
            radius=8,
            color=colour,
            fill=True,
            fill_color=colour,
            popup=f'{area}: {diff:+.0f} ({pct:+.2f}%)'
        ).add_to(m)
    return m

year1=widgets.Dropdown(options=years,description='Year 1',value=years[0])
year2=widgets.Dropdown(options=years,description='Year 2',value=years[-1])
out=widgets.Output()

def update(change=None):
    with out:
        out.clear_output()
        display(create_map(year1.value,year2.value))

year1.observe(update,'value')
year2.observe(update,'value')
display(widgets.VBox([widgets.HBox([year1,year2]),out]))
update()
